In [1]:
from trainer import Trainer
from dataset import get_ogb_data
from torch_geometric import seed_everything
from ogb.nodeproppred import Evaluator
from predict import evaluate_test
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm

/data/home/spektor/miniconda3/envs/baskin/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d_name = 'arxiv'
data, split_idx, num_classes = get_ogb_data('arxiv')
evaluator = Evaluator(name=f'ogbn-{d_name}')

Number of nodes in the graph: 169343
Number of edges in the graph: 1166243
Number of training nodes: 90941
Number of validation nodes: 29799
Number of test nodes: 48603
Node feature matrix with shape: torch.Size([169343, 128])
Graph connectivity in COO format with shape: torch.Size([2, 1166243])
Target to train against : torch.Size([169343, 1])
Node feature length 128
number of target categories: 40


In [3]:
DEEP_DEPTH = 16
SHALLOW_DEPTH = 2 
depths_dict = {'deep': [DEEP_DEPTH], 'shallow': [SHALLOW_DEPTH], 'both': [DEEP_DEPTH, SHALLOW_DEPTH]}
gnn_model_types = ['GCN', 'GAT', 'SAGE']
epochs = 250
cfg={
    "optimizer_type": 'adam',
    "model_type": 'SAGE',
    "lr_cfg":{
        "lr": 0.01,
    },
    "train_log_step": 10,
    "save_interval": 50
}

In [ ]:
# train
for depth, depths_list in depths_dict.items():
    for model_type in gnn_model_types:
        cfg['model_type'] = model_type
        trainer = Trainer(cfg=cfg, model_dir=f'{depth}_{model_type}_temp')
        trainer.run(data, split_idx, num_classes, evaluator, depths_list, epochs)